In [6]:
from gbmi.exp_modular_fine_tuning.train import MODULAR_ADDITION_113_CLOCK_CONFIG
from gbmi.exp_modular_fine_tuning.train import ModularFineTuningTrainingWrapper
from gbmi.model import train_or_load_model
import torch
from torch import tensor
from math import *

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
config = MODULAR_ADDITION_113_CLOCK_CONFIG
runtime, model = train_or_load_model(config)

wandb:   1 of 1 files downloaded.  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [14]:
loss_fn = ModularFineTuningTrainingWrapper.loss_fn
for param in model.parameters():
    param.requires_grad = False
embedding_matrix = model.embed.W_E.detach().clone()
embedding_matrix.requires_grad = False
model.embed = torch.nn.Identity()

In [15]:
p = 113
q = 2*p
class DifferentModClock(torch.nn.Module):
    def __init__(self,q):
        super(DifferentModClock,self).__init__()
        self.W_e = torch.nn.Parameter(-0.5/sqrt(q+1) + torch.rand((p+1,q+1))/sqrt(q+1)).to('cuda')
        self.W_u = torch.nn.Parameter(-0.5/sqrt(p) + torch.rand((q,p))/sqrt(p)).to('cuda')
    def forward(self,x):
        z = torch.nn.functional.one_hot(x.to('cuda')).to(torch.float).T
        y = z.T @ (self.W_e).T @ embedding_matrix
        y = y.unsqueeze(0)
        return self.W_u @ (model(y)[0].T)
Clock = DifferentModClock(q)
print(Clock(torch.tensor([1,2,226])))

tensor([[-1.1551e+01, -4.1845e+00, -3.5317e+00],
        [-5.0986e+00, -2.3193e+00, -1.0811e+00],
        [-2.8107e+00, -6.3421e-01, -5.2152e-02],
        [-7.4410e+00, -3.0344e+00, -1.9548e+00],
        [-8.8225e+00, -2.4522e+00, -1.4526e+00],
        [-2.0526e+00, -1.2787e+00,  2.4912e-01],
        [ 6.6056e-01,  5.9062e-01, -1.2428e+00],
        [ 1.0250e+01,  2.9770e+00,  2.6668e+00],
        [-5.2383e+00, -2.2974e+00, -2.6732e+00],
        [ 1.0726e-01, -2.8095e-01, -4.4778e-01],
        [-3.5795e+00, -4.0874e-01, -1.7823e+00],
        [-4.1496e-01, -1.1167e+00, -1.3078e+00],
        [ 6.5487e+00,  1.8415e+00,  1.4953e+00],
        [ 4.3439e+00,  2.7943e+00,  2.3933e+00],
        [-2.4677e+00, -7.9783e-01, -7.5480e-01],
        [-1.1743e+00, -2.4960e-01,  9.5912e-01],
        [-1.3703e+00, -8.8560e-01, -4.0773e-01],
        [ 2.5356e-01,  1.1326e+00,  5.8724e-01],
        [-5.5122e-01, -5.3896e-01, -6.6659e-01],
        [-1.4553e+00, -2.3928e-01, -4.2061e-01],
        [ 2.9958e-01

In [16]:
def run_batch(x,  prefix):

        x = x[0]
        labels = (x[:, 0] + x[:, 1]) % q
        y_preds = Clock(x)
        loss = loss_fn(y_preds, labels)
        self.log(f"{prefix}loss", loss, prog_bar=True)
        return loss